In [ ]:
import os 
os.chdir(r'C:\Users\PRAVEEN\Desktop\dl_cnn_classifier')

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data:Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [7]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [8]:
class ConfigurationManager():
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifact_root])

    def get_trainingconfig(self)-> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"PetImages")
        create_directories([
            Path(training.root_dir),

        ])


        training_config = TrainingConfig(
                root_dir=Path(training.root_dir),
                trained_model_path =Path(training.trained_model_path),
                updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
                training_data=Path(training_data),
                params_epochs= params.EPOCHS,
                params_batch_size=params.BATCH_SIZE,
                params_is_augmentation= params.AUGMENTATION,
                params_image_size= params.IMAGE_SIZE,
        )
        return  training_config 

In [ ]:
from configparser import Interpolation
import tensorflow as tf

class Training:
    def __init__(self,config:TrainingConfig):
        self.config=config

    def get_base_model(self,):
        self.model=tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.20
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size,
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmentation:
            training_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range= 0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            training_datagenerator=valid_datagenerator
        self.train_generator=training_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='trainig',
            shuffle=True,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)

    def train(self,callback_list:list):
        self.steps_per_epoch=self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps=self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epochs=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )
        self.save_model(
            path=self.config.trained_model_path,
            medel=self.model
        )



In [ ]:
try:  
    config=ConfigurationManager()
    prepare_callback_config=config.get_prepare_callback_config()
    prepare_callback=PrepareCallback(config=prepare_callback_config)
    callback_list=prepare_callback.get_tb_ckpt_callbacks()

    training_config=config.get_trainingconfig()
    training=Training(training_config)
    training.get_base_model()
    training.train_valid_generator()
    trianing.train(
        
    )
except Exception as e:
    raise e